# Post processing of minian output

In [ ]:
cd "C:/Users/Manip2/SCRIPTS/CodePythonAudrey/CodePythonAurelie/HayLabAnalysis/minian"

In [ ]:
#%%capture
#%load_ext autoreload
#%autoreload 2
minian_path = "."
import os

import sys
import warnings
import itertools as itt
import numpy as np
import xarray as xr
import holoviews as hv
import pandas as pd
from pathlib import Path
from holoviews.operation.datashader import datashade, regrid
from dask.diagnostics import ProgressBar
sys.path.append(minian_path)

from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)
from minian.visualization import AlignViewer
import matplotlib.pyplot as plt
from pathlib import Path


import warnings
warnings.filterwarnings("ignore")

load faulty minian dataset

In [ ]:
dpath = Path("//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/Purple/session2/V4_Miniscope/session23")
minian_ds_path_Origin = os.path.join(dpath, "minian")
minian_ds_path_Final = os.path.join(dpath, "minian2")

def correct_meta(ds,  minian_ds_path_Origin):
    path_obj =Path(minian_ds_path_Origin)
    if len(path_obj.parts)==12: # Not a DrugExperiment with Subsessions       
        ds['session'] = path_obj.parents[1].name
        ds['animal'] = path_obj.parents[3].name
    elif len(path_obj.parts)==11: # Not a DrugExperiment with No Subsessions
        ds['session'] = path_obj.parents[1].name
        ds['animal'] = path_obj.parents[2].name
    elif len(path_obj.parts)==13: # DrugExperiment with Subsessions
        ds['session'] = path_obj.parents[0].name
        ds['animal'] = path_obj.parents[4].name
    print(f"['animal: {ds['animal'].values}']")
    return ds

minian_ds = open_minian(minian_ds_path_Origin, post_process=correct_meta)

A = minian_ds['A']
b = minian_ds['b']
b0 = minian_ds['b0']
C = minian_ds['C']
c0 = minian_ds['c0']
f = minian_ds['f']
max_proj = minian_ds['max_proj']
motion = minian_ds['motion']
S = minian_ds['S']

param_save_minian = {
    "dpath": minian_ds_path_Final,
    "meta_dict": dict(session=-1, animal=-2),
    "overwrite": True
}

A = save_minian(A.rename("A"), **param_save_minian)
b = save_minian(b.rename("b"), **param_save_minian)
b0 = save_minian(b0.rename("b0"), **param_save_minian)
C = save_minian(C.rename("C"), **param_save_minian)
c0 = save_minian(c0.rename("c0"), **param_save_minian)
f = save_minian(f.rename("f"), **param_save_minian)
max_proj = save_minian(max_proj.rename("max_proj"), **param_save_minian)
motion = save_minian(motion.rename("motion"), **param_save_minian)
S = save_minian(S.rename("S"), **param_save_minian)

In [ ]:
def find_session_folders(root_path):
    sessions = []
    sessions_path=[]
    # Iterate through items in the root_path
    for item in os.listdir(root_path):
        item_path = os.path.join(root_path, item)
        if os.path.isdir(item_path):
            # Check if the directory name contains "session"
            if "session" in item:
                sessions.append(item)
                sessions_path.append(item_path)
            else:
                # Check the subdirectories of the current directory
                for sub_item in os.listdir(item_path):
                    sub_item_path = os.path.join(item_path, sub_item)
                    if os.path.isdir(sub_item_path) and "session" in sub_item:
                        sessions.append(sub_item)
                        sessions_path.append(sub_item_path)
                        
    return sessions, sessions_path

def correct_meta(ds,  minian_ds_path_Origin):
    path_obj =Path(minian_ds_path_Origin)
    if len(path_obj.parts)==12: # Not a DrugExperiment with Subsessions       
        ds['session'] = path_obj.parents[1].name
        ds['animal'] = path_obj.parents[3].name
    elif len(path_obj.parts)==11: # Not a DrugExperiment with No Subsessions
        ds['session'] = path_obj.parents[1].name
        ds['animal'] = path_obj.parents[2].name
    elif len(path_obj.parts)==13: # DrugExperiment with Subsessions
        ds['session'] = path_obj.parents[0].name
        ds['animal'] = path_obj.parents[4].name
    print(f"['animal: {ds['animal'].values}']")
    return ds

In [ ]:
MiceList=['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK','Purple' ,'ThreeColDotsOK']
for mice in MiceList:
    dpath0 = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/" 
    dpath1=dpath0 + mice
    folder_base = Path(dpath1)

    sessions = []
    subsessions = []
    nb_minian_total=0
    dict_Calcium = {}
    dict_Spike = {}
    dict_Scoring = {}
    dict_Stamps = {}
    dict_TodropFile = {}
    dict_StampsMiniscope = {}
    dict_Path={}

    sessions, sessions_path = find_session_folders(folder_base)
    nb_sessions=len(sessions)

    for sess,session in enumerate(sessions):  
        session_path=Path(sessions_path[sess])
        folder_mini = session_path / f'V4_Miniscope'
        nb_subsessions = sum(1 for p in folder_mini.iterdir() if p.is_dir() and p.name.startswith("session"))

        if nb_subsessions!=0:
            for x in range(1, nb_subsessions+1):    
                subsession= session + str(x)
                subsessions.append(subsession)    
                dpath=folder_mini / subsession
                print(dpath)

                minian_ds_path_Origin = os.path.join(dpath, "minian")
                minian_ds_path_Final = os.path.join(dpath, "minian2")

                minian_ds = open_minian(minian_ds_path_Origin, post_process=correct_meta)

                A = minian_ds['A']
                b = minian_ds['b']
                b0 = minian_ds['b0']
                C = minian_ds['C']
                c0 = minian_ds['c0']
                f = minian_ds['f']
                max_proj = minian_ds['max_proj']
                motion = minian_ds['motion']
                S = minian_ds['S']

                param_save_minian = {
                    "dpath": minian_ds_path_Final,
                    "meta_dict": dict(session=-1, animal=-2),
                    "overwrite": True
                }

                A = save_minian(A.rename("A"), **param_save_minian)
                b = save_minian(b.rename("b"), **param_save_minian)
                b0 = save_minian(b0.rename("b0"), **param_save_minian)
                C = save_minian(C.rename("C"), **param_save_minian)
                c0 = save_minian(c0.rename("c0"), **param_save_minian)
                f = save_minian(f.rename("f"), **param_save_minian)
                max_proj = save_minian(max_proj.rename("max_proj"), **param_save_minian)
                motion = save_minian(motion.rename("motion"), **param_save_minian)
                S = save_minian(S.rename("S"), **param_save_minian)
        else:
            dpath=folder_mini 
            print(dpath)
            minian_ds_path_Origin = os.path.join(dpath, "minian")
            minian_ds_path_Final = os.path.join(dpath, "minian2")

            minian_ds = open_minian(minian_ds_path_Origin, post_process=correct_meta)

            A = minian_ds['A']
            b = minian_ds['b']
            b0 = minian_ds['b0']
            C = minian_ds['C']
            c0 = minian_ds['c0']
            f = minian_ds['f']
            max_proj = minian_ds['max_proj']
            motion = minian_ds['motion']
            S = minian_ds['S']

            param_save_minian = {
                "dpath": minian_ds_path_Final,
                "meta_dict": dict(session=-1, animal=-2),
                "overwrite": True
            }

            A = save_minian(A.rename("A"), **param_save_minian)
            b = save_minian(b.rename("b"), **param_save_minian)
            b0 = save_minian(b0.rename("b0"), **param_save_minian)
            C = save_minian(C.rename("C"), **param_save_minian)
            c0 = save_minian(c0.rename("c0"), **param_save_minian)
            f = save_minian(f.rename("f"), **param_save_minian)
            max_proj = save_minian(max_proj.rename("max_proj"), **param_save_minian)
            motion = save_minian(motion.rename("motion"), **param_save_minian)
            S = save_minian(S.rename("S"), **param_save_minian)

This one is to flip only the map so that cross-registration is possible

In [ ]:
dpath = Path("//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/ThreeColDotsOK/CGP/sessionB/V4_Miniscope/sessionB3")
minian_ds_path_Origin = os.path.join(dpath, "minian")

param_save_minian = {
    "dpath": minian_ds_path_Origin,
    "meta_dict": dict(session=-1, animal=-2),
    "overwrite": True
}

def correct_meta(ds,  minian_ds_path_Origin):
    path_obj =Path(minian_ds_path_Origin)
    if len(path_obj.parts)==12: # Not a DrugExperiment with Subsessions       
        ds['session'] = path_obj.parents[1].name
        ds['animal'] = path_obj.parents[3].name
    elif len(path_obj.parts)==11: # Not a DrugExperiment with No Subsessions
        ds['session'] = path_obj.parents[1].name
        ds['animal'] = path_obj.parents[2].name
    elif len(path_obj.parts)==13: # DrugExperiment with Subsessions
        ds['session'] = path_obj.parents[0].name
        ds['animal'] = path_obj.parents[4].name
    print(f"['animal: {ds['animal'].values}']")
    return ds

minian_ds = open_minian(minian_ds_path_Origin, post_process=correct_meta)

max_proj = minian_ds['max_proj']
max_projnp = minian_ds['max_proj'].values

"""
max_projnp = np.rot90(max_projnp) # rotate 90°degrees clockwise
max_projnp = np.rot90(max_projnp) # rotate 90°degrees clockwise
max_projnp = np.rot90(max_projnp) # rotate 90°degrees clockwise

max_projnp = np.flip(max_projnp, (0,1)) #flip horizontally & vertically
"""
max_proj.values = max_projnp
max_proj = save_minian(max_proj.rename("max_proj"), **param_save_minian)
